### 학습 목표

- 초대용량 데이터는 어떻게 처리할까요?


- 데이터를 분할하여 불러오고 처리하는 방법에 대해서 연습해봅니다.


- 불량 검출 문제에 대해 살펴봅니다.


- Class imbalance 문제를 해결하는 방법에 대해 알아봅니다.

## Bosch Production Line Performance

- 공정과정 데이터를 통해 제품 내부 불량 검출하기

- 주어진 공정변수를 이용하여 정상/불량 분류하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_cat_part = pd.read_csv("../input/bosch-production-line-performance/train_categorical.csv.zip",
                            nrows=1000)
train_numeric_part = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                                nrows=1000)
train_date_part = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip",
                             nrows=1000)
print(train_cat_part.shape, train_numeric_part.shape, train_date_part.shape)

#### 대용량 데이터가 메인 메모리에 한번에 적재가 안될때!

1. 사용할 column을 몇개 불러온다. (usecols)

2. 데이터를 쪼개서 임시로 불러와서 계산하고, 다시 불러온다. (chunksize)

In [ ]:
train_date_part # start_station, end_station

### train_date 데이터를 요약해봅시다.

- 각 row마다 측정이 처음된 station 번호와, 마지막으로 측정이 된 station 번호를 찾습니다.

In [ ]:
row = train_date_part.loc[0]
row[row.index.str.contains("S2_")] # 0.01 == 6분

# 1. station별 column 찾기 : 1157개 -> 52개
counts = train_date_part.count() # column이 index로 들어가 있는 Series를 만들려고.
date_cols = counts.reset_index()["index"].str.split("_", expand=True)
col_idx = date_cols.drop_duplicates(1).index  # column 1을 기준으로 중복 제거
date_cols = train_date_part.columns[col_idx]
date_cols

In [ ]:
row = train_date_part.loc[999]
row[row.index.str.contains("S51_")] # 0.01 == 6분

In [ ]:
# 2. start_station, end_station 찾기

train_date = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip",
                        usecols=date_cols)

## 전체 데이터에서 column 순서대로 scan하면서 처음으로 NaN이 아닌 column의 station이 start_station,
## 마지막까지 scan하면서 마지막으로 NaN이 아니었던 column의 station이 end_station.
train_date["start_station"] = -1
train_date["end_station"] = -1

for col in train_date.drop(columns=["Id", "start_station", "end_station"]).columns:
    notnulls = ~train_date[col].isnull() # null이 아닌 row들의 위치
    station_name = int(col.split("_")[1][1:])
    
    train_date.loc[(notnulls) & (train_date.start_station == -1), "start_station"] = station_name
    train_date.loc[(notnulls), "end_station"] = station_name
    
train_date

In [ ]:
train_date = train_date[["Id", "start_station", "end_station"]]
train_date

### 차원의 저주(Curse of Dimensionality)

- 머신러닝 모델이 데이터가 존재하는 공간이 너무 고차원일 때, <=> feature(column)가 너무 많을 때

- 머신러닝 모델이 예측력을 잃어버리는 문제. (성능이 저하)


- 해결책

> 차원을 줄입니다! (= column수를 줄인다)

> 1. feature selection : 분석에 사용할 column을 직접 고른다.
>> 결측치가 많은 column을 버리거나, 도메인 지식으로 필요한 변수를 고르거나, ....

> 2. feature extraction : dimensionality reduction method(e.g. PCA)를 사용한다.

In [ ]:
missing_ratio = pd.Series(index=train_numeric_part.columns,
                         data=np.zeros(len(train_numeric_part.columns))) # 970
column_means = pd.Series(index=train_numeric_part.columns,
                         data=np.zeros(len(train_numeric_part.columns)))
length = 0

for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                         chunksize=100000):
    #display(chunk.isnull().sum())
    temp = chunk.isnull().sum()  # chunk별 결측치 합
    temp2 = chunk.sum()
    
    length = length + len(chunk)
    missing_ratio = missing_ratio + temp  # 결측치 누적합
    column_means = column_means + temp2  # 실제 값의 누적합
    #display(missing_ratio + chunk.isnull().sum())
    #break

display(missing_ratio) # 결측치 개수
display(column_means / length)

In [ ]:
length

In [ ]:
missing_ratio = missing_ratio / length  # 비율로 변환

In [ ]:
# 결측치가 50%를 넘는 column을 지우겠다.
train_numeric_part.drop(columns=train_numeric_part.columns[missing_ratio > 0.5])

In [ ]:
# 결측 비율이 50%를 넘지 않는 column들만 뽑아오겠다.
usecols = train_numeric_part.columns[missing_ratio <= 0.5]
usecols

In [ ]:
# train_numeric_part2 = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
#                                 usecols=["Id", "Response"]) # 불러온 column을 지정.
# train_numeric_part2

In [ ]:
# train_numeric_part2.info()

In [ ]:
# # 불량품 비율
# #train_numeric_part.Response.value_counts() # 99.5 : 0.05
# sns.countplot(data=train_numeric_part, x="Response")  # target value distribution
# ## Class imbalance problem, 클래스 불균형 문제!

### 1. 대용량 데이터 다루기

Technique1. 잘라서 불러오기


Technique2. Reducing Memory Technique (dtype 변경하기)

In [ ]:
# %%timeit
# X = pca.fit_transform(data)

In [ ]:
# pca = PCA(n_components=0.90) # 전체 데이터의 90% 정보를 보존하는 저차원 공간으로 변환해주세요.
# X = pca.fit_transform(data)
# pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, X.shape[1]+1)],
#                      data=X)
# display(pca_df)

In [ ]:
from sklearn.decomposition import PCA
data = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                        usecols=usecols)
data = data.fillna(0) # 결측치를 0으로 채움
ids = data.Id
responses = data.Response
data = data.drop(columns=["Id", "Response"])
pca = PCA(n_components=6)
X = pca.fit_transform(data)
pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, 7)],
                     data=X)
display(pca_df)

In [ ]:
## 데이터를 합쳐줍니다.
# pca_df (numeric 데이터로 만든 데이터) + train_date(date 데이터로 만든 데이터)
pca_df["Id"] = ids
pca_df["Response"] = responses
X = pd.merge(train_date, pca_df, on="Id")
X

In [ ]:
# # Technique 1. PCA (ML) / AutoEncoder (DL)
# from sklearn.decomposition import PCA

# for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
#                         usecols=usecols,
#                         chunksize=10000):
#     #display(chunk) # dropna()
#     chunk = chunk.fillna(0) # 결측치를 0으로 채움
#     temp = chunk.drop(columns=["Id", "Response"])
#     pca = PCA(n_components=6)
#     X = pca.fit_transform(temp)
#     pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, 7)],
#                          data=X)
#     display(pca_df)
#     break

### dtype

- np.int8, np.int16, np.int32, np.int64 : 정수 하나를 몇 bit로 표현할 것인가?

> int8 : -2^7 ~ 2^7 - 1  --> -128 ~ 127  --> 00000000 ~ 11111111

> int16 : -2^15 ~ 2^15 - 1 --> -65536 ~ 65535 --> 0000000000000000 ~ 11111111111111111111

...

- np.float32 / np.float64 : 실수 하나를 몇 bit로 표현할 것인가?
 (single precision / double precision)

> 실수 표현은 항상 근사치를 쓰기 때문에, 얼마나 자세하게 표현할지의 차이!

> 필연적으로 발생하는 수치적 차이를 "numerical error"라고 부릅니다.

In [ ]:
# Technique 2. dtype 변경
pca_df.info()

In [ ]:
pca_df.astype(np.float32).info() # float64 -> float32로 메모리 반토막.

In [ ]:
# ## Reducing DataFrame memory size by ~65%
# #####
# def reduce_mem_usage(props):
#     start_mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage of properties dataframe is :",start_mem_usg," MB")
#     NAlist = [] # Keeps track of columns that have missing values filled in. 
#     for col in props.columns:
#         if props[col].dtype != object:  # Exclude strings
            
#             # Print current column type
#             print("******************************")
#             print("Column: ",col)
#             print("dtype before: ",props[col].dtype)
            
#             # make variables for Int, max and min
#             IsInt = False
#             mx = props[col].max()
#             mn = props[col].min()
            
#             # Integer does not support NA, therefore, NA needs to be filled
#             if not np.isfinite(props[col]).all(): 
#                 NAlist.append(col)
#                 props[col].fillna(mn-1,inplace=True)  
#             # test if column can be converted to an integer
#             asint = props[col].fillna(0).astype(np.int64)
#             result = (props[col] - asint)
#             result = result.sum()
#             if result > -0.01 and result < 0.01:
#                 IsInt = True

            
#             # Make Integer/unsigned Integer datatypes
#             if IsInt:
#                 if mn >= 0:
#                     if mx < 255:
#                         props[col] = props[col].astype(np.uint8)
#                     elif mx < 65535:
#                         props[col] = props[col].astype(np.uint16)
#                     elif mx < 4294967295:
#                         props[col] = props[col].astype(np.uint32)
#                     else:
#                         props[col] = props[col].astype(np.uint64)
#             else:
#                     if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
#                         props[col] = props[col].astype(np.int8)
#                     elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
#                         props[col] = props[col].astype(np.int16)
#                     elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
#                         props[col] = props[col].astype(np.int32)
#                     elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
#                         props[col] = props[col].astype(np.int64)    
            
#             # Make float datatypes 32 bit
#             else:
#                 props[col] = props[col].astype(np.float32)
            
#             # Print new column type
#             print("dtype after: ",props[col].dtype)
#             print("******************************")
    
#     # Print final result
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
#     return props, NAlist

### 2. 불량 검출 문제 다루기 (Class Imbalance problem)

Technique 1. Sampling Method (Under / Over)
> binary classification (이진 분류) : 정상이냐 불량이냐로 예측하는 문제!

> 일반적으로 불량에 해당하는 케이스가 매우매우 적습니다. (minority class)

> 일반적으로 정상에 해당하는 케이스가 아주아주 많습니다. (majority class)

> 머신러닝 분류 문제에서는 클래스의 비율이 비슷할수록 학습이 잘됩니다.

>> minority -> majority (oversampling)

>> majority -> minority (undersampling)




Technique 2. Change to Outlier Detection problem

- 주류(또는 대다수) : majority

- 주류가 아닌 데이터를 outlier(이상치)라고 판단하는 방법.

- IQR, IsolationForest (ML), DBSCAN (ML)





(Advanced) Technique 3. Semi-supervised Learning (DL), MakinaRocks

In [ ]:
# Technique 1.
# X : feature vector (독립변수), 학습 데이터(문제)
# y : target value   (종속변수), 목표 값 (정답)
X = pca_df
y = responses
print(X.shape, y.shape)

## 1-1. Undersampling - y에서 1의 개수로 0인 데이터를 줄이는 것. (99.5 --> 0.05)
normal = X[y == 0] # response가 0인 데이터
failure = X[y == 1] # response가 1인 데이터
print(normal.shape, failure.shape) ## 1176868 --> 6879

normal_under = normal.sample(n=failure.shape[0], random_state=0xC0FFEE) # 불량 데이터 개수만큼 샘플링!
print(normal_under.shape, failure.shape)

In [ ]:
## 1-2. Oversampilng - y가 1인 데이터를 y가 0인 데이터의 개수로 늘리는 것. (0.05 --> 99.5)
from imblearn.over_sampling import SMOTE

sm = SMOTE()
# X, y 데이터를 가지고 y에서 자동으로 minority class를 찾은 뒤에
# majority class의 크기와 같게 oversampling 해주는 함수.
X_oversampled, y_oversampled = sm.fit_resample(X, y)
print(X_oversampled.shape, y_oversampled.shape)

In [ ]:
failure.shape[0]

In [ ]:
#X_oversampled[y_oversampled == 0] # == normal
X_oversampled[y_oversampled == 1][6879:] # 새로 생긴 데이터.

In [ ]:
## 1-3. Hybrid Approach : Undersampling + Oversampling

### 정상 데이터를 500,000개로 undersampling한 뒤, 불량 데이터를 oversampling을 수행하여
### 데이터를 1,000,000개가 되도록 만들어주세요!!

X_under = normal.sample(n=500000).reset_index(drop=True)
y_under = np.zeros(len(X_under))
X_total = pd.concat([X_under, failure]).reset_index(drop=True)
y_total = np.hstack([y_under, np.ones(len(failure))]).astype(int)


# SMOTE에 정상과 불량 데이터가 합쳐진 전체 데이터를 넣으면 알아서, (기준은 항상 y)
# majority class 크기에 맞게 oversampling이 됩니다.
X_hybrid, y_hybrid = sm.fit_resample(X_total, y_total)
print(X_hybrid.shape, y_hybrid.shape)

In [ ]:
# write csv file
pca_df.to_csv("PCA_data.csv", index=False)

In [ ]:
import pickle

# Data dump
with open("pca_data.pk", "wb") as f:
    pickle.dump(pca_df, f)
    
# Data load
with open("pca_data.pk", "rb") as f:
    pca_df = pickle.load(f)

In [ ]:
pca_df

In [ ]:
# Technique 2. IQR
#sns.boxplot(train_numeric_part.L0_S0_F0)
#sns.boxplot(data=train_numeric_part, x="L0_S0_F0")

def IQR_outlier(data, column):
    temp = data[column].dropna() ## 결측치가 없어야함!
    q25 = np.percentile(temp, 25) # 25%
    q75 = np.percentile(temp, 75) # 75%
    IQR = q75 - q25 # Inter-Quartile Range
    lower_bound = q25 - IQR*1.5
    upper_bound = q75 + IQR*1.5
    outlier = temp.loc[(temp < lower_bound) | (temp > upper_bound)]
    return outlier

In [ ]:
sns.boxplot(data=train_numeric_part, x="L0_S0_F0")

In [ ]:
IQR_outlier(train_numeric_part, "L0_S0_F0")

In [ ]:
# from sklearn.cluster import DBSCAN

# db = DBSCAN(n_jobs=-1)
# pred = db.fit_predict(X)
# pred

In [ ]:
# X[pred == -1] # outlier

### if, ML

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

